In [9]:
#!pip install --user panoptes-client

In [24]:
import os
from glob import glob
from panoptes_client import Panoptes, Project, SubjectSet, Subject
from astropy.utils.console import ProgressBar

In [2]:
client = Panoptes.connect(username=os.environ.get('ZOONIVERSE_USER'), password=os.environ.get('ZOONIVERSE_PASSWORD'))

In [3]:
project = Project.find(slug='ebellm/alert-production-zoo')

In [29]:
indir = 'hsc-cosmos_stamp_images'

In [13]:
project.links

In [15]:
print([(ss.id, ss.display_name) for ss in project.links.subject_sets])

[('80863', 'HiTS')]


In [16]:
# delete old ones if desired
#subject_set = SubjectSet.find(80863)
#for subject in subject_set.subjects:
#    subject_set.remove(subject.id)

In [14]:
for workflow in project.links.workflows:
    print(workflow.display_name)

Scan cutouts


In [27]:
# make a new subject set
subject_set = SubjectSet()

subject_set.links.project = project
subject_set.display_name = 'HSC-COSMOS'

subject_set.save()

{'subject_sets': [{'id': '81026',
   'display_name': 'HSC-COSMOS',
   'set_member_subjects_count': 0,
   'metadata': {},
   'created_at': '2020-03-10T17:32:34.084Z',
   'updated_at': '2020-03-10T17:32:34.084Z',
   'href': '/subject_sets/81026',
   'links': {'project': '11152', 'workflows': [], 'subjects': []}}],
 'links': {'subject_sets.project': {'href': '/projects/{subject_sets.project}',
   'type': 'projects'},
  'subject_sets.workflows': {'href': '/workflows?subject_set_id={subject_sets.id}',
   'type': 'workflows'}},
 'meta': {'subject_sets': {'page': 1,
   'page_size': 20,
   'count': 1,
   'include': [],
   'page_count': 1,
   'previous_page': None,
   'next_page': None,
   'first_href': '/subject_sets',
   'previous_href': None,
   'next_href': None,
   'last_href': '/subject_sets'}}}

In [28]:
project.reload()

In [30]:
files = glob(f'./{indir}/*.png')

In [31]:
len(files)

1000

In [32]:
def get_id(filename):
    return int(filename.split('/')[-1].split('_')[0])

In [33]:
get_id(files[0])

50809763759392069

In [34]:
new_subjects = []

with ProgressBar(len(files), ipython_widget=True) as bar:

    for filename in files:
        subject = Subject()

        subject.links.project = project
        subject.add_location(filename)

        subject.metadata.update({'diaSourceId':get_id(filename)})

        subject.save()
        new_subjects.append(subject)
        bar.update()
subject_set.add(new_subjects)

FloatProgress(value=0.0)

In [35]:
workflow = project.links.workflows[0]
workflow.links.subject_sets.add(subject_set)